In [3]:
import numpy as np
import pandas as pd
from datetime import datetime
from tensorflow.keras.models import load_model
import pickle
import matplotlib.pyplot as plt
import time

In [4]:
series = pd.read_csv('Final Dataset.csv')
series['Datetime'] = pd.to_datetime(series['Datetime'])
series=series.set_index('Datetime')
series_daily=series.resample('D').sum()


In [5]:
prophet = pickle.load(open('prophetwebmodel.pkl', 'rb'))

In [6]:
holt = pickle.load(open('holtwebmodel.pkl', 'rb'))


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(series_daily)
scaled_train = scaler.transform(series_daily)

n_input = 30
n_features=1
lstmmodel = load_model('lstmwebmodel.hdf5')

In [8]:
!pip install flask-ngrok
!pip install flask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!pip install pyngrok
!ngrok authtoken '2AvPU6DunTpBu2HmCpqH7DaxG18_3as1rok8KHVYFy7SysngY'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 8.6 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=4b44b046301e85a207e3ced9c586f1df47c584a54095cd563f0b86c62675691e
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, url_for, render_template, request, flash

app = Flask(__name__)
run_with_ngrok(app)
@app.route('/')
def home():
    return render_template("index.html")


@app.route('/prediction',methods=['POST','GET'])
def prediction():
    date_format = "%Y-%m-%d"
    ourdate = datetime.strptime(
    request.form['date'],date_format)
    datasetEndDate = datetime.strptime('2019-10-31',date_format)
    delta = ourdate-datasetEndDate
    period = delta.days
    predictions = holt.forecast(period)
    op = predictions[period-1]

    mydate = ourdate.strftime('%Y-%m-%d')
    predict_df = list()
    predict_df.append(mydate)
    predict_df = pd.DataFrame(predict_df)
    predict_df.columns = ['ds']
    predict_df['ds'] = pd.to_datetime(mydate,yearfirst=True)
    forecast = prophet.predict(predict_df)
    forecast = forecast['yhat'][0]

    our_predictions = []
    first_eval_batch = scaled_train[-n_input:]
    current_batch = first_eval_batch.reshape((1, n_input, n_features))
    for i in range(period):
      
      current_pred = lstmmodel.predict(current_batch)[0]
      
      our_predictions.append(current_pred) 
      
      current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

    true_predictions = scaler.inverse_transform(our_predictions)
    lstm = true_predictions[-1]


    data = {'Holt':op, 'Prophet':forecast, 'LSTM':lstm}
    models = list(data.keys())
    values = list(data.values())
  
    fig = plt.figure(figsize = (10, 5))
 
    # creating the bar plot
    plt.bar(models, values, color ='maroon',
        width = 0.4)
    def add_value_label(x_list,y_list):
        for i in range(len(x_list)):
            plt.text(i,y_list[i],str(y_list[i]), ha="center")
    add_value_label(models,values)
    
 
    plt.xlabel("Models")
    plt.ylabel("Power in Megawatts")
    plt.title("Model comparisons")
    plt.savefig('/content/drive/MyDrive/savedmodels/plot.png')

    outputtext = 'The power needed on {} is :{}MegaWatts using holtWintersModel \n  {}MegaWatts using prophetModel \n and {}Megawatts using RNNLSTM'.format(ourdate,op,forecast,lstm)
    text = outputtext.split('\n')
    time.sleep(20)
    
    


    
    return render_template('index.html', prediction_text=text,name = 'new_plot', url ='https://drive.google.com/uc?export=view&id=1oOBa1ITED1H-xiaGynGqkGzZg6gH-3tf')




if __name__ == '__main__':
    app.run() 




 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://66d8-34-125-167-121.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [13/Jul/2022 09:32:32] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jul/2022 09:32:33] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jul/2022 09:32:33] "GET /static/Typing_light_01_Videvo.webm HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [13/Jul/2022 09:32:37] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.7/dist-packages/numpy/lib/stride_tricks.py:536: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  args = [np.array(_m, copy=False, subok=subok) for _m in args]
INFO:werkzeug:127.0.0.1 - - [13/Jul/2022 09:33:42] "POST /prediction HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jul/2022 09:33:47] "GET /static/Typing_light_01_Videvo.webm HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [13/Jul/2022 09:34:57

In [ ]:
# run_with_ngrok(app)
# app.run()